In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
odds_f = pd.read_csv("bettingdata_odds.csv", parse_dates=["datetime"], index_col="datetime")

In [3]:
odds_f.index

DatetimeIndex(['2019-01-02', '2019-01-02', '2019-01-02', '2019-01-02',
               '2019-01-02', '2019-01-02', '2019-01-02', '2019-01-02',
               '2019-01-02', '2019-01-03',
               ...
               '2022-03-26', '2022-03-26', '2022-03-26', '2022-03-27',
               '2022-03-27', '2022-03-27', '2022-03-27', '2022-03-27',
               '2022-03-27', '2022-03-27'],
              dtype='datetime64[ns]', name='datetime', length=4321, freq=None)

In [4]:
odds_f.drop(datetime(2019, 2, 17), axis=0, inplace=True)
odds_f.drop(datetime(2022, 2, 20), axis=0, inplace=True)
odds_f.drop(datetime(2020, 2, 16), axis=0, inplace=True)
odds_f.drop(datetime(2021, 3, 7), axis=0, inplace=True)

In [5]:
predict_f = pd.read_csv("predictions/five_thirtyeight/nba_elo.csv", parse_dates=["date"], index_col="date")

In [9]:
odds_f_sub = odds_f[["AwayTeam", "HomeTeam","HomeTeamHasWon","ConsensusHomeMoneyLine", "ConsensusAwayMoneyLine"]]
predict_f_sub = predict_f[["team1", "team2", "elo_prob1", "elo_prob2"]]
predict_f_sub = predict_f_sub[predict_f_sub.index >= odds_f_sub.index[0]]
predict_f_sub = predict_f_sub[predict_f_sub.index <= odds_f_sub.index[-1]]

In [10]:
predict_f_sub

,team1,team2,elo_prob1,elo_prob2
date,,,,
2019-01-02,CHO,DAL,0.671360,0.328640
2019-01-02,CLE,MIA,0.421730,0.578270
2019-01-02,WAS,ATL,0.724603,0.275397
2019-01-02,BRK,NOP,0.536187,0.463813
2019-01-02,CHI,ORL,0.552163,0.447837
...,...,...,...,...
2022-03-27,WAS,GSW,0.347830,0.652170
2022-03-27,BOS,MIN,0.766541,0.233459
2022-03-27,NOP,LAL,0.755368,0.244632


In [11]:
odds_f_sub

,AwayTeam,HomeTeam,HomeTeamHasWon,ConsensusHomeMoneyLine,ConsensusAwayMoneyLine
datetime,,,,,
2019-01-02,ORL,CHI,False,102.0,-112.0
2019-01-02,OKC,LAL,False,190.0,-210.0
2019-01-02,PHI,PHO,False,150.0,-160.0
2019-01-02,DET,MEM,False,-240.0,220.0
2019-01-02,MIN,BOS,True,-250.0,230.0
...,...,...,...,...,...
2022-03-27,PHI,PHO,True,-185.0,152.0
2022-03-27,GS,WAS,True,226.0,-281.0
2022-03-27,LAL,NO,True,-173.0,143.0


In [12]:
#combine_f = predict_f_sub.merge(odds_f_sub, left_index=True,right_index=True, how="inner")

In [13]:
odds_f_sub["AwayTeam"].unique()

array(['ORL', 'OKC', 'PHI', 'DET', 'MIN', 'NO', 'DAL', 'ATL', 'MIA',
       'HOU', 'DEN', 'TOR', 'UTA', 'IND', 'WAS', 'BKN', 'LAC', 'NY', 'GS',
       'CHA', 'MEM', 'LAL', 'SA', 'SAC', 'CLE', 'MIL', 'CHI', 'PHO',
       'BOS', 'POR'], dtype=object)

In [14]:
odds_f_sub_teams = np.union1d(odds_f_sub["HomeTeam"].unique(), odds_f_sub["AwayTeam"].unique())

In [15]:
predict_f_sub_teams  = np.union1d( predict_f_sub["team1"].unique(), predict_f_sub["team2"].unique())

In [16]:
[x for x in odds_f_sub_teams if x not in predict_f_sub_teams]

['BKN', 'CHA', 'GS', 'NO', 'NY', 'SA']

In [17]:
[x for x in predict_f_sub_teams if x not in odds_f_sub_teams]

['BRK', 'CHO', 'GSW', 'NOP', 'NYK', 'SAS']

In [18]:
TEAM_NAME_MAPPING = {"NO": "NOP",
                     "BKN": "BRK",
                     "NY": "NYK",
                     "GS": "GSW",
                     "CHA": "CHO",
                     "SA": "SAS"}

In [19]:
odds_f["AwayTeam"] = odds_f["AwayTeam"].apply(lambda x: TEAM_NAME_MAPPING[x] if x in TEAM_NAME_MAPPING.keys() else x)
odds_f["HomeTeam"] = odds_f["HomeTeam"].apply(lambda x: TEAM_NAME_MAPPING[x] if x in TEAM_NAME_MAPPING.keys() else x)

In [20]:
odds_f_sub = odds_f[["AwayTeam", "HomeTeam","HomeTeamHasWon","ConsensusHomeMoneyLine", "ConsensusAwayMoneyLine"]]

In [28]:
predict_f_sub

,team1,team2,elo_prob1,elo_prob2
date,,,,
2019-01-02,CHO,DAL,0.671360,0.328640
2019-01-02,CLE,MIA,0.421730,0.578270
2019-01-02,WAS,ATL,0.724603,0.275397
2019-01-02,BRK,NOP,0.536187,0.463813
2019-01-02,CHI,ORL,0.552163,0.447837
...,...,...,...,...
2022-03-27,WAS,GSW,0.347830,0.652170
2022-03-27,BOS,MIN,0.766541,0.233459
2022-03-27,NOP,LAL,0.755368,0.244632


,AwayTeam,HomeTeam,HomeTeamHasWon,ConsensusHomeMoneyLine,ConsensusAwayMoneyLine
datetime,,,,,
2019-01-02,ORL,CHI,False,102.0,-112.0
2019-01-02,OKC,LAL,False,190.0,-210.0
2019-01-02,PHI,PHO,False,150.0,-160.0
2019-01-02,DET,MEM,False,-240.0,220.0
2019-01-02,MIN,BOS,True,-250.0,230.0
...,...,...,...,...,...
2022-03-27,PHI,PHO,True,-185.0,152.0
2022-03-27,GSW,WAS,True,226.0,-281.0
2022-03-27,LAL,NOP,True,-173.0,143.0


In [33]:
combine_f = predict_f_sub.reset_index().merge(odds_f_sub.reset_index(), left_on=['date', 'team1', 'team2'], right_on=['datetime', 'HomeTeam', 'AwayTeam'], how="inner")

In [34]:
combine_f

,date,team1,team2,elo_prob1,elo_prob2,datetime,AwayTeam,HomeTeam,HomeTeamHasWon,ConsensusHomeMoneyLine,ConsensusAwayMoneyLine
0,2019-01-02,CHO,DAL,0.671360,0.328640,2019-01-02,DAL,CHO,False,-117.0,107.0
1,2019-01-02,CLE,MIA,0.421730,0.578270,2019-01-02,MIA,CLE,False,210.0,-230.0
2,2019-01-02,WAS,ATL,0.724603,0.275397,2019-01-02,ATL,WAS,True,-240.0,220.0
3,2019-01-02,BRK,NOP,0.536187,0.463813,2019-01-02,NOP,BRK,True,115.0,-125.0
4,2019-01-02,CHI,ORL,0.552163,0.447837,2019-01-02,ORL,CHI,False,102.0,-112.0
...,...,...,...,...,...,...,...,...,...,...,...
4188,2022-03-27,WAS,GSW,0.347830,0.652170,2022-03-27,GSW,WAS,True,226.0,-281.0
4189,2022-03-27,BOS,MIN,0.766541,0.233459,2022-03-27,MIN,BOS,True,-259.0,208.0
4190,2022-03-27,NOP,LAL,0.755368,0.244632,2022-03-27,LAL,NOP,True,-173.0,143.0
4191,2022-03-27,DAL,UTA,0.621091,0.378909,2022-03-27,UTA,DAL,True,-168.0,139.0


In [39]:
combine_f.rename(columns={"elo_prob1": "HomeTeamProb", "elo_prob2": "AwayTeamProb"}, inplace=True)

In [40]:
combine_f.columns

Index(['date', 'team1', 'team2', 'HomeTeamProb', 'AwayTeamProb', 'datetime',
       'AwayTeam', 'HomeTeam', 'HomeTeamHasWon', 'ConsensusHomeMoneyLine',
       'ConsensusAwayMoneyLine'],
      dtype='object')

In [44]:
combine_f.drop(["team1", "team2", "date"], axis=1, inplace= True)

In [46]:
combine_f.set_index("datetime", inplace=True)

In [47]:
combine_f

,HomeTeamProb,AwayTeamProb,AwayTeam,HomeTeam,HomeTeamHasWon,ConsensusHomeMoneyLine,ConsensusAwayMoneyLine
datetime,,,,,,,
2019-01-02,0.671360,0.328640,DAL,CHO,False,-117.0,107.0
2019-01-02,0.421730,0.578270,MIA,CLE,False,210.0,-230.0
2019-01-02,0.724603,0.275397,ATL,WAS,True,-240.0,220.0
2019-01-02,0.536187,0.463813,NOP,BRK,True,115.0,-125.0
2019-01-02,0.552163,0.447837,ORL,CHI,False,102.0,-112.0
...,...,...,...,...,...,...,...
2022-03-27,0.347830,0.652170,GSW,WAS,True,226.0,-281.0
2022-03-27,0.766541,0.233459,MIN,BOS,True,-259.0,208.0
2022-03-27,0.755368,0.244632,LAL,NOP,True,-173.0,143.0


In [48]:
combine_f.to_csv("combine_data.csv")